In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lmfit
import scipy.constants
from scipy.signal import savgol_filter
import hvplot.pandas

%run "C:\Users\conde\Desktop\My Archives\Codes\Python\Data Analysis\my_optical_analysis_functions.ipynb"

In [ ]:
directory = "C:\\Users\\conde\\Desktop\\My Archives\\Photonicamp\\Artigos Publicados\\LTOI e LN on Sapphire\\LTOI"
directory = "C:\\Users\\conde\\Desktop\\My Archives\\Photonicamp\\Artigos Publicados\\On-Chip Backward Stimulated Brillouin Scattering in Lithium Niobate Waveguides\\Data\\Latest_Spectrums"

directory_file_list = os.listdir(directory)
selected_files = []
counter = 0

for i in directory_file_list:
    if i.endswith(".csv") or i.endswith(".parq"):
        print(f"{counter}: {i}")
        selected_files.append(i)
        counter += 1

In [ ]:
idx_file = 52
header = 21

file_name = selected_files[idx_file]
df_raw = pd.read_csv(directory+"\\"+selected_files[idx_file], header=header)
#df_raw = pd.read_parquet(directory+"\\"+selected_files[idx_file])
columns_raw=df_raw.columns

print(file_name)
#df_raw.hvplot.explorer(cmap='viridis', rasterize=True)

In [4]:
idx_crop_i = int(0)
idx_crop_f = int(-1)
point_jump = 50
idx_data_column = 1
idx_MZI_column = 2
idx_HCN_column = 2
flip = False
#----------

data_crop = (df_raw[columns_raw[idx_data_column]].values)[idx_crop_i:idx_crop_f][::point_jump]
MZI_crop = (df_raw[columns_raw[idx_MZI_column]].values)[idx_crop_i:idx_crop_f][::point_jump]
HCN_crop = (df_raw[columns_raw[idx_HCN_column]].values)[idx_crop_i:idx_crop_f][::point_jump]

if flip:
    data_crop = np.flip(data_crop)
    MZI_crop = np.flip(MZI_crop)
    HCN_crop = np.flip(HCN_crop)

In [ ]:
MZI_normalized, MZI_baseline_up, MZI_baseline_down = background_flattening(MZI_crop, lam=1000*len(MZI_crop), baseline_up=True, baseline_down=True)

%matplotlib inline
fig, ax = plt.subplots(2,2,figsize=(13,4))
fig.tight_layout()

ax[0,0].plot(MZI_crop)
ax[0,0].plot(MZI_baseline_up, c='black')
ax[0,0].plot(MZI_baseline_down, c='black')
ax[0,0].grid(True)

ax[0,1].plot(MZI_normalized)
ax[0,1].grid(True)

idx_MZI_plot_i = 40000
idx_MZI_plot_f = 50000
ax[1,0].plot(MZI_crop[idx_MZI_plot_i:idx_MZI_plot_f])
ax[1,0].plot(MZI_baseline_up[idx_MZI_plot_i:idx_MZI_plot_f], c='black')
ax[1,0].plot(MZI_baseline_down[idx_MZI_plot_i:idx_MZI_plot_f], c='black')
ax[1,0].grid(True)

idx_MZI_plot_offset = 10000
idx_MZI_plot_i = idx_MZI_plot_i + idx_MZI_plot_offset
idx_MZI_plot_f = idx_MZI_plot_f + idx_MZI_plot_offset
ax[1,1].plot(MZI_crop[idx_MZI_plot_i:idx_MZI_plot_f])
ax[1,1].plot(MZI_baseline_up[idx_MZI_plot_i:idx_MZI_plot_f], c='black')
ax[1,1].plot(MZI_baseline_down[idx_MZI_plot_i:idx_MZI_plot_f], c='black')
ax[1,1].grid(True)

print(file_name)

In [ ]:
filt = False
if filt:
    MZI_filtered = savgol_filter(MZI_normalized, window_length=2, polyorder=1, mode='interp')
else:
    MZI_filtered = MZI_normalized

%matplotlib inline
fig, ax = plt.subplots(1,2,figsize=(13,2))
fig.tight_layout()

ax[0].plot(MZI_normalized, linewidth=3)
ax[0].plot(MZI_filtered, linewidth=1)
ax[0].set_ylabel("MZI Filt vs MZI Crop")
ax[0].grid(True)

idx_MZI_plot_i = 40000
idx_MZI_plot_f = 50000
ax[1].plot(MZI_normalized[idx_MZI_plot_i:idx_MZI_plot_f], linewidth=3)
ax[1].plot(MZI_filtered[idx_MZI_plot_i:idx_MZI_plot_f], linewidth=1)
ax[1].grid(True)

print(file_name)

In [ ]:
FSR = 137.0e6 # in HZ
detuning, data, MZI, idx_MZI_peaks = from_MZI_to_detuning(data_crop, MZI_filtered, FSR=FSR, distance=200, height=0.2, centralize=True)

%matplotlib inline
fig, ax = plt.subplots(2,1,figsize=(13,4))
fig.tight_layout()

ax[0].plot(detuning, MZI, linewidth=1)
ax[0].scatter(detuning[idx_MZI_peaks], MZI[idx_MZI_peaks], c='red', zorder=5)
ax[0].set_ylabel("MZI Norm.")
ax[0].set_xlabel("Detuning [units of FSR]")
ax[0].grid(True)

ax[1].plot(detuning, data, linewidth=3)
ax[1].set_xlim(detuning[0], detuning[-1])
ax[1].set_ylabel("Data")
ax[1].set_xlabel("Detuning [Hz]")
ax[1].grid(True)

print(file_name)

In [ ]:
data_normalized, data_baseline_up, data_baseline_down = background_flattening(data, lam=1000000*len(data), baseline_up=False, baseline_down=False)
data_normalized_peaks_height = 0.8
idx_data_normalized_peaks = find_peaks(data_normalized, height=data_normalized_peaks_height, distance=1000)[0]
#idx_data_normalized_peaks = find_peaks(1-data_normalized, height=data_normalized_peaks_height, distance=1000)[0]

%matplotlib inline
fig, ax = plt.subplots(1,2,figsize=(13,4))
fig.tight_layout()

ax[0].plot(data_normalized)
ax[0].plot([0, len(data_normalized)], [data_normalized_peaks_height, data_normalized_peaks_height], c="black", linestyle="dashed")
ax[0].scatter(idx_data_normalized_peaks, data_normalized[idx_data_normalized_peaks], c='red')
ax[0].grid(True)
counter=0
for i in idx_data_normalized_peaks:
    ax[0].annotate(str(counter) + " - " + str(i), xy=(i, data_normalized[i]), size=7, rotation=-90)
    counter += 1

ax[1].plot(detuning, data)
# ax[1].plot(detuning, data_baseline_up, c='black')
# ax[1].plot(detuning, data_baseline_down, c='black')
ax[1].scatter(detuning[idx_data_normalized_peaks], data[idx_data_normalized_peaks], c='red')
ax[1].grid(True)
counter=0
for i in idx_data_normalized_peaks:
    ax[1].annotate(str(counter) + " - " + str(i), xy=(detuning[i], data[i]), size=7, rotation=-90)
    counter += 1

print(file_name)

In [ ]:
PXA_AnalogOut_Conversion = 192.66 #dB/V
PXA_AnalogOut_at_1V = -10 #dBm
PXA_Attenuation = 40 #dB

def from_V_to_dBm_PXA_AnalogOut(y):
    return PXA_AnalogOut_Conversion*(y-1) + PXA_AnalogOut_at_1V - PXA_Attenuation

X_AS = detuning_centralized[:idx_data_peaks[1]]
Y_AS = from_V_to_dBm_PXA_AnalogOut(data[:idx_data_peaks[1]])
idx_AS_fiber_peak = np.argmax(Y_AS)

X_S = detuning_centralized[idx_data_peaks[1]:]
Y_S = from_V_to_dBm_PXA_AnalogOut(data[idx_data_peaks[1]:])
idx_S_fiber_peak = np.argmax(Y_S)

fig, ax = plt.subplots(1,2,figsize=(13,4))
fig.tight_layout()

ax[0].plot(X_AS/1e9,Y_AS)
ax[0].scatter(X_AS[idx_AS_fiber_peak]/1e9, Y_AS[idx_AS_fiber_peak], c='red')
ax[0].annotate(str(np.round(X_AS[idx_AS_fiber_peak]/1e9,2))+"[GHz]", xy=(X_AS[idx_AS_fiber_peak]/1e9, Y_AS[idx_AS_fiber_peak]), size=7, rotation=0)
ax[0].set_xlabel("Frequency [GHz]")
ax[0].grid(True)

ax[1].plot(X_S/1e9,Y_S)
ax[1].scatter(X_S[idx_S_fiber_peak]/1e9, Y_S[idx_S_fiber_peak], c='red')
ax[1].annotate(str(np.round(X_S[idx_S_fiber_peak]/1e9,2))+"[GHz]", xy=(X_S[idx_S_fiber_peak]/1e9, Y_S[idx_S_fiber_peak]), size=7, rotation=0)
ax[1].set_xlabel("Frequency [GHz]")
ax[1].grid(True)

print(file_name)

In [ ]:
freq_i = 10.0e9
freq_f = 12.0e9

idx_AS_freq_i = np.argmin(abs(X_AS + freq_i))
idx_AS_freq_f = np.argmin(abs(X_AS + freq_f))
X_AS_crop = X_AS[idx_AS_freq_f:idx_AS_freq_i]
Y_AS_crop = Y_AS[idx_AS_freq_f:idx_AS_freq_i]

idx_S_freq_i = np.argmin(abs(X_S - freq_i))
idx_S_freq_f = np.argmin(abs(X_S - freq_f))
X_S_crop = X_S[idx_S_freq_i:idx_S_freq_f]
Y_S_crop = Y_S[idx_S_freq_i:idx_S_freq_f]

fig, ax = plt.subplots(1,2,figsize=(13,4))
fig.tight_layout()

ax[0].plot(X_AS_crop, Y_AS_crop)
ax[0].set_xlabel("Frequency [Hz]")
ax[0].grid(True)

ax[1].plot(X_S_crop, Y_S_crop)
ax[1].set_xlabel("Frequency [Hz]")
ax[1].grid(True)

print(file_name)

In [10]:
N = 3
side = "AS"
fi_guess = [-7.4, -7.5, -7.8]
δfi_guess = [0.1, 0.2, 0.1]
Pi_guess = [-104, -112, -112]
background_guess = -127

In [ ]:
if side == "AS":
  X = X_AS_crop/1e9
  Y = Y_AS_crop
  fiber_signal_dBm = np.max(Y_AS)
elif side == "S":
  X = X_S_crop/1e9
  Y = Y_S_crop
  fiber_signal_dBm = np.max(Y_S)

if N==1:
  def lorentzians_dBm(f, f0, δf0, P0, background):
    Ω = 2*np.pi*f
    Ωm0 = 2*np.pi*f0
    Γm0 = 2*np.pi*δf0
    P0_mW = 10**(P0/10)
    return 10*np.log10(P0_mW*((Γm0/2)**2)/((Ω - Ωm0)**2 + (Γm0/2)**2) + 10**(background/10))
  
  model = lmfit.Model(lorentzians_dBm)
  fit = model.fit(Y, f=X,
                    f0=fi_guess[0],
                    δf0=δfi_guess[0],
                    P0=Pi_guess[0],
                    background=background_guess)
  
  χ2_fit = fit.chisqr
  fi_fit = fit.params["f0"].value
  δfi_fit = fit.params["δf0"].value
  Pi_fit = fit.params["P0"].value
  background_fit = fit.params["background"].value
  Qm = abs(fi_fit/δfi_fit)
  legend_fit = [
    'Ωm/2π = '    + str(np.round(fi_fit, 3)) + ' [GHz] \n' \
    'Γm/2π = '    + str(np.round(δfi_fit*1e3, 3)) + ' [MHz] \n' \
    'Qm = '    + str(np.round(Qm, 3)) + ' \n' \
    'P = '    + str(np.round(Pi_fit, 3)) + ' [dBm] \n\n' \
    'background = '    + str(np.round(background_fit, 3)) + ' [dBm]' \
  ]

  fig, ax = plt.subplots(figsize=(12,4))
  fig.tight_layout()

  ax.plot(X, Y, color='black', linewidth=2)
  ax.plot(X, lorentzians_dBm(X, fi_fit, δfi_fit, Pi_fit, background_fit), color='red', linewidth=2, linestyle='dashed')
  ax.plot(X, lorentzians_dBm(X, fi_fit, δfi_fit, Pi_fit, background_fit-999), color='green', linewidth=2, linestyle='dashed')
  ax.plot(X, lorentzians_dBm(X, 0*fi_fit, 0*δfi_fit, Pi_fit-999, background_fit), color='blue', linewidth=2, linestyle='dashed')
  ax.set_xlabel("Frequency [GHz]")
  ax.legend(legend_fit)
  ax.grid(True)

elif N==2:
  def lorentzians_dBm(f, f0, δf0, P0, f1, δf1, P1, background):
    Ω = 2*np.pi*f
    Ωm0 = 2*np.pi*f0
    Ωm1 = 2*np.pi*f1
    Γm0 = 2*np.pi*δf0
    Γm1 = 2*np.pi*δf1
    P0_mW = abs(10**(P0/10))
    P1_mW = abs(10**(P1/10))
    return  10*np.log10(P0_mW*((Γm0/2)**2)/((Ω - Ωm0)**2 + (Γm0/2)**2) + P1_mW*((Γm1/2)**2)/((Ω - Ωm1)**2 + (Γm1/2)**2) + 10**(background/10))
  
  model = lmfit.Model(lorentzians_dBm)
  fit = model.fit(Y, f=X,
                        f0=fi_guess[0], f1=fi_guess[1],
                        δf0=δfi_guess[0], δf1=δfi_guess[1],
                        P0=Pi_guess[0], P1=Pi_guess[1],
                        background=background_guess)
  
  χ2_fit = fit.chisqr
  fi_fit = [fit.params["f0"].value, fit.params["f1"].value]
  δfi_fit = [fit.params["δf0"].value, fit.params["δf1"].value]
  Pi_fit = [fit.params["P0"].value, fit.params["P1"].value]
  background_fit = fit.params["background"].value
  Qm = [abs(fi_fit[0]/δfi_fit[0]), abs(fi_fit[1]/δfi_fit[1])]
  legend_fit = [
    'Ωm0/2π = '    + str(np.round(fi_fit[0], 3)) + ' [GHz] \n' \
    'Γm0/2π = '    + str(np.round(δfi_fit[0]*1e3, 3)) + ' [MHz] \n' \
    'Qm0 = '    + str(np.round(Qm[0], 3)) + ' \n' \
    'P0 = '    + str(np.round(Pi_fit[0], 3)) + ' [dBm] \n\n' \
    'Ωm1/2π = '    + str(np.round(fi_fit[1], 3)) + ' [GHz] \n' \
    'Γm1/2π = '    + str(np.round(δfi_fit[1]*1e3, 3)) + ' [MHz] \n' \
    'Qm1 = '    + str(np.round(Qm[1], 3)) + ' \n' \
    'P1 = '    + str(np.round(Pi_fit[1], 3)) + ' [dBm] \n\n' \
    'background = '    + str(np.round(background_fit, 3)) + ' [dBm]' \
  ]

  fig, ax = plt.subplots(figsize=(12,4))
  fig.tight_layout()

  ax.plot(X, Y, color='black', linewidth=2)
  ax.plot(X, lorentzians_dBm(X, fi_fit[0], δfi_fit[0], Pi_fit[0],
                                fi_fit[1], δfi_fit[1], Pi_fit[1], background_fit), color='red', linewidth=2, linestyle='dashed')
  
  ax.plot(X, lorentzians_dBm(X, fi_fit[0], δfi_fit[0], Pi_fit[0],
                                0*fi_fit[1], 0*δfi_fit[1], Pi_fit[1]-999, background_fit-999), color='green', linewidth=2, linestyle='dashed')
  
  ax.plot(X, lorentzians_dBm(X, 0*fi_fit[0], 0*δfi_fit[0], Pi_fit[0]-999,
                                fi_fit[1], δfi_fit[1], Pi_fit[1], background_fit-999), color='green', linewidth=2, linestyle='dashed')
  
  ax.plot(X, lorentzians_dBm(X, 0*fi_fit[0], 0*δfi_fit[0], Pi_fit[0]-999,
                                0*fi_fit[1], 0*δfi_fit[1], Pi_fit[1]-999, background_fit), color='blue', linewidth=2, linestyle='dashed')
  
  ax.set_xlabel("Frequency [GHz]")
  ax.legend(legend_fit)
  ax.grid(True)
  
elif N==3:
  def lorentzians_dBm(f, f0, δf0, P0, f1, δf1, P1, f2, δf2, P2, background):
    Ω = 2*np.pi*f
    Ωm0 = 2*np.pi*f0
    Ωm1 = 2*np.pi*f1
    Ωm2 = 2*np.pi*f2
    Γm0 = 2*np.pi*δf0
    Γm1 = 2*np.pi*δf1
    Γm2 = 2*np.pi*δf2
    P0_mW = abs(10**(P0/10))
    P1_mW = abs(10**(P1/10))
    P2_mW = abs(10**(P2/10))
    return  10*np.log10(P0_mW*((Γm0/2)**2)/((Ω - Ωm0)**2 + (Γm0/2)**2) + P1_mW*((Γm1/2)**2)/((Ω - Ωm1)**2 + (Γm1/2)**2) + P2_mW*((Γm2/2)**2)/((Ω - Ωm2)**2 + (Γm2/2)**2) + 10**(background/10))
  
  model = lmfit.Model(lorentzians_dBm)
  fit = model.fit(Y, f=X,
                        f0=fi_guess[0], f1=fi_guess[1], f2=fi_guess[2],
                        δf0=δfi_guess[0], δf1=δfi_guess[1], δf2=δfi_guess[2],
                        P0=Pi_guess[0], P1=Pi_guess[1], P2=Pi_guess[2],
                        background=background_guess)
  
  χ2_fit = fit.chisqr
  fi_fit = [fit.params["f0"].value, fit.params["f1"].value, fit.params["f2"].value]
  δfi_fit = [fit.params["δf0"].value, fit.params["δf1"].value, fit.params["δf2"].value]
  Pi_fit = [fit.params["P0"].value, fit.params["P1"].value, fit.params["P2"].value]
  background_fit = fit.params["background"].value
  Qm = [abs(fi_fit[0]/δfi_fit[0]), abs(fi_fit[1]/δfi_fit[1]), abs(fi_fit[2]/δfi_fit[2])]
  legend_fit = [
    'Ωm0/2π = '    + str(np.round(fi_fit[0], 3)) + ' [GHz] \n' \
    'Γm0/2π = '    + str(np.round(δfi_fit[0]*1e3, 3)) + ' [MHz] \n' \
    'Qm0 = '    + str(np.round(Qm[0], 3)) + ' \n' \
    'P0 = '    + str(np.round(Pi_fit[0], 3)) + ' [dBm] \n\n' \
    'Ωm1/2π = '    + str(np.round(fi_fit[1], 3)) + ' [GHz] \n' \
    'Γm1/2π = '    + str(np.round(δfi_fit[1]*1e3, 3)) + ' [MHz] \n' \
    'Qm1 = '    + str(np.round(Qm[1], 3)) + ' \n' \
    'P1 = '    + str(np.round(Pi_fit[1], 3)) + ' [dBm] \n\n' \
    'Ωm2/2π = '    + str(np.round(fi_fit[2], 3)) + ' [GHz] \n' \
    'Γm2/2π = '    + str(np.round(δfi_fit[2]*1e3, 3)) + ' [MHz] \n' \
    'Qm2 = '    + str(np.round(Qm[2], 3)) + ' \n' \
    'P2 = '    + str(np.round(Pi_fit[2], 3)) + ' [dBm] \n\n' \
    'background = '    + str(np.round(background_fit, 3)) + ' [dBm]' \
  ]

  fig, ax = plt.subplots(figsize=(12,4))
  fig.tight_layout()

  ax.plot(X, Y, color='black', linewidth=2)
  ax.plot(X, lorentzians_dBm(X, fi_fit[0], δfi_fit[0], Pi_fit[0],
                              fi_fit[1], δfi_fit[1], Pi_fit[1],
                              fi_fit[2], δfi_fit[2], Pi_fit[2], background_fit), color='red', linewidth=2, linestyle='dashed')
  
  ax.plot(X, lorentzians_dBm(X, fi_fit[0], δfi_fit[0], Pi_fit[0],
                              0*fi_fit[1], 0*δfi_fit[1], Pi_fit[1]-999,
                              0*fi_fit[2], 0*δfi_fit[2], Pi_fit[2]-999, background_fit-999), color='green', linewidth=2, linestyle='dashed')
  
  ax.plot(X, lorentzians_dBm(X, 0*fi_fit[0], 0*δfi_fit[0], Pi_fit[0]-999,
                              fi_fit[1], δfi_fit[1], Pi_fit[1],
                              0*fi_fit[2], 0*δfi_fit[2], Pi_fit[2]-999, background_fit-999), color='green', linewidth=2, linestyle='dashed')
  
  ax.plot(X, lorentzians_dBm(X, 0*fi_fit[0], 0*δfi_fit[0], Pi_fit[0]-999,
                              0*fi_fit[1], 0*δfi_fit[1], Pi_fit[1]-999,
                              fi_fit[2], δfi_fit[2], Pi_fit[2], background_fit-999), color='green', linewidth=2, linestyle='dashed')
  
  ax.plot(X, lorentzians_dBm(X, 0*fi_fit[0], 0*δfi_fit[0], Pi_fit[0]-999,
                              0*fi_fit[1], 0*δfi_fit[1], Pi_fit[1]-999,
                              0*fi_fit[2], 0*δfi_fit[2], Pi_fit[2]-999, background_fit), color='blue', linewidth=2, linestyle='dashed')
  
  ax.set_xlabel("Frequency [GHz]")
  ax.legend(legend_fit)
  ax.grid(True)
  
print(file_name)

In [13]:
angle = int(file_name.split('_')[2][:-10])
if angle == -60:
    L_straight = 6200e-6
elif angle == -55:
    L_straight = 5848e-6
elif angle == -50:
    L_straight = 5544e-6
elif angle == -45:
    L_straight = 5337e-6
elif angle == -40:
    L_straight = 5125e-6
elif angle == -35:
    L_straight = 4906e-6
elif angle == -30:
    L_straight = 4750e-6
elif angle == -25:
    L_straight = 4595e-6
elif angle == -20:
    L_straight = 4560e-6
elif angle == -15:
    L_straight = 3842e-6
elif angle == -10:
    L_straight = 3818e-6
elif angle == -5:
    L_straight = 3893e-6
elif angle == 0:
    L_straight = 3800e-6
elif angle == 5:
    L_straight = 3693e-6
elif angle == 10:
    L_straight = 3778e-6
elif angle == 15:
    L_straight = 3842e-6
elif angle == 20:
    L_straight = 3970e-6
elif angle == 25:
    L_straight = 4025e-6
elif angle == 30:
    L_straight = 4280e-6
elif angle == 35:
    L_straight = 4506e-6
elif angle == 40:
    L_straight = 4725e-6
elif angle == 45:
    L_straight = 5037e-6
elif angle == 50:
    L_straight = 5344e-6
elif angle == 55:
    L_straight = 5648e-6
elif angle == 60:
    L_straight = 6100e-6

In [ ]:
Tp = 0.19
αp = 25.78
L1 = 9.8
if angle==0 or angle==60 or angle==-60:   
    L2_mode1 = L_straight + 5000e-6
    L2_mode2 = L_straight + 5000e-6
    L2_mode3 = L_straight + 5000e-6
else:   
    L2_mode1 = 5000e-6
    L2_mode2 = L_straight + 5000e-6
    L2_mode3 = 5000e-6
L3 = 6
GB_fiber = 0.36

if N==1:
    peak_signal_1 = lorentzians_dBm(X, fi_fit, δfi_fit, Pi_fit, background_fit)
    GB1_LN = GB_fiber*((L1 + Tp*Tp*L3*np.exp(-αp*L2_mode1))/(Tp*L2_mode1))*(10**((peak_signal_1-fiber_signal_dBm)/10))
    
    GB_LN = GB1_LN
    GB_Qm_LN = GB1_LN/Qm[0]

    fig, ax = plt.subplots(2,1,figsize=(14,5))
    fig.tight_layout()

    ax[0].plot(X, GB1_LN)
    ax[0].plot(X, GB_LN)
    idx_GB1_max = np.argmax(GB1_LN)
    ax[0].annotate(str(np.round(GB_LN[idx_GB1_max],3)), xy=(X[idx_GB1_max], GB_LN[idx_GB1_max]), size=10, rotation=0)
    ax[0].scatter(X[idx_GB1_max], GB_LN[idx_GB1_max], c='red')
    ax[0].grid(True)

    ax[1].plot(X, GB1_LN/Qm)
    ax[1].plot(X, GB_Qm_LN)
    ax[1].grid(True)

elif N==2:
    peak_signal_1 = lorentzians_dBm(X, fi_fit[0], δfi_fit[0], Pi_fit[0], 0*fi_fit[1], 0*δfi_fit[1], Pi_fit[1]-999, background_fit)
    GB1_LN = GB_fiber*((L1 + Tp*Tp*L3*np.exp(-αp*L2_mode1))/(Tp*L2_mode1))*(10**((peak_signal_1-fiber_signal_dBm)/10))

    peak_signal_2 = lorentzians_dBm(X, 0*fi_fit[0], 0*δfi_fit[0], Pi_fit[0]-999, fi_fit[1], δfi_fit[1], Pi_fit[1], background_fit)
    GB2_LN = GB_fiber*((L1 + Tp*Tp*L3*np.exp(-αp*L2_mode2))/(Tp*L2_mode2))*(10**((peak_signal_2-fiber_signal_dBm)/10))
    
    GB_LN = GB1_LN + GB2_LN
    GB_Qm_LN = GB1_LN/Qm[0] + GB2_LN/Qm[1]

    fig, ax = plt.subplots(2,1,figsize=(14,5))
    fig.tight_layout()

    ax[0].plot(X, GB1_LN)
    ax[0].plot(X, GB2_LN)
    ax[0].plot(X, GB_LN)
    idx_GB1_max = np.argmax(GB1_LN)
    idx_GB2_max = np.argmax(GB2_LN)
    ax[0].annotate(str(np.round(GB_LN[idx_GB1_max],3)), xy=(X[idx_GB1_max], GB_LN[idx_GB1_max]), size=10, rotation=0)
    ax[0].annotate(str(np.round(GB_LN[idx_GB2_max],3)), xy=(X[idx_GB2_max], GB_LN[idx_GB2_max]), size=10, rotation=0)
    ax[0].scatter(X[idx_GB1_max], GB_LN[idx_GB1_max], c='red')
    ax[0].scatter(X[idx_GB2_max], GB_LN[idx_GB2_max], c='red')
    ax[0].grid(True)

    ax[1].plot(X, GB1_LN/Qm[0])
    ax[1].plot(X, GB2_LN/Qm[1])
    ax[1].plot(X, GB_Qm_LN)
    ax[1].grid(True)

elif N==3:
    peak_signal_1 = lorentzians_dBm(X, fi_fit[0], δfi_fit[0], Pi_fit[0], 0*fi_fit[1], 0*δfi_fit[1], Pi_fit[1]-999, 0*fi_fit[2], 0*δfi_fit[2], Pi_fit[2]-999, background_fit)
    GB1_LN = GB_fiber*((L1 + Tp*Tp*L3*np.exp(-αp*L2_mode1))/(Tp*L2_mode1))*(10**((peak_signal_1-fiber_signal_dBm)/10))

    peak_signal_2 = lorentzians_dBm(X, 0*fi_fit[0], 0*δfi_fit[0], Pi_fit[0]-999, fi_fit[1], δfi_fit[1], Pi_fit[1], 0*fi_fit[2], 0*δfi_fit[2], Pi_fit[2]-999, background_fit)
    GB2_LN = GB_fiber*((L1 + Tp*Tp*L3*np.exp(-αp*L2_mode2))/(Tp*L2_mode2))*(10**((peak_signal_2-fiber_signal_dBm)/10))

    peak_signal_3 = lorentzians_dBm(X, 0*fi_fit[0], 0*δfi_fit[0], Pi_fit[0]-999, 0*fi_fit[1], 0*δfi_fit[1], Pi_fit[1]-999, fi_fit[2], δfi_fit[2], Pi_fit[2], background_fit)
    GB3_LN = GB_fiber*((L1 + Tp*Tp*L3*np.exp(-αp*L2_mode3))/(Tp*L2_mode3))*(10**((peak_signal_3-fiber_signal_dBm)/10))

    GB_LN = GB1_LN + GB2_LN + GB3_LN
    GB_Qm_LN = GB1_LN/Qm[0] + GB2_LN/Qm[1] + GB3_LN/Qm[2]

    fig, ax = plt.subplots(2,1,figsize=(14,5))
    fig.tight_layout()

    ax[0].plot(X, GB1_LN)
    ax[0].plot(X, GB2_LN)
    ax[0].plot(X, GB3_LN)
    ax[0].plot(X, GB_LN)
    idx_GB1_max = np.argmax(GB1_LN)
    idx_GB2_max = np.argmax(GB2_LN)
    idx_GB3_max = np.argmax(GB3_LN)
    ax[0].annotate(str(np.round(GB_LN[idx_GB1_max],3)), xy=(X[idx_GB1_max], GB_LN[idx_GB1_max]), size=10, rotation=0)
    ax[0].annotate(str(np.round(GB_LN[idx_GB2_max],3)), xy=(X[idx_GB2_max], GB_LN[idx_GB2_max]), size=10, rotation=0)
    ax[0].annotate(str(np.round(GB_LN[idx_GB3_max],3)), xy=(X[idx_GB3_max], GB_LN[idx_GB3_max]), size=10, rotation=0)
    ax[0].scatter(X[idx_GB1_max], GB_LN[idx_GB1_max], c='red')
    ax[0].scatter(X[idx_GB2_max], GB_LN[idx_GB2_max], c='red')
    ax[0].scatter(X[idx_GB3_max], GB_LN[idx_GB3_max], c='red')
    ax[0].set_ylabel("GB [1/W/m]")
    ax[0].grid(True)

    ax[1].plot(X, GB1_LN/Qm[0])
    ax[1].plot(X, GB2_LN/Qm[1])
    ax[1].plot(X, GB3_LN/Qm[2])
    ax[1].plot(X, GB_Qm_LN)
    ax[1].set_ylabel("GB/Qm [1/W/m]")
    ax[1].grid(True)
    
print(file_name)